# Handoff 与 Input Filter 示例

本 Notebook 演示 OpenAI Agents SDK 的 **Handoff（任务转移）** 机制，以及如何通过 **input_filter** 在转移时过滤对话历史。

## 工作流程概览

1. **first_agent**：通用助手，带有 `random_number_tool`
2. **second_agent**：主入口，当用户说西班牙语时 handoff 到 Spanish Assistant
3. **spanish_agent**：仅用西班牙语回复的助手

当 handoff 发生时，`spanish_handoff_message_filter` 会：
- 移除所有工具相关消息（function_call、tool_output 等）
- 移除历史中的前两条消息（演示用）

## 1. 导入与定义

In [1]:
import json
import random

from agents import Agent, HandoffInputData, Runner, function_tool, handoff, trace
from agents.extensions import handoff_filters
from agents.models import is_gpt_5_default


@function_tool
def random_number_tool(max: int) -> int:
    """Return a random integer between 0 and the given maximum."""
    return random.randint(0, max)


def spanish_handoff_message_filter(handoff_message_data: HandoffInputData) -> HandoffInputData:
    if is_gpt_5_default():
        print("gpt-5 is enabled, so we're not filtering the input history")
        return HandoffInputData(
            input_history=handoff_message_data.input_history,
            pre_handoff_items=tuple(handoff_message_data.pre_handoff_items),
            new_items=tuple(handoff_message_data.new_items),
        )

    # 移除所有工具相关消息
    handoff_message_data = handoff_filters.remove_all_tools(handoff_message_data)

    # 移除历史中的前两条（演示用）
    history = (
        tuple(handoff_message_data.input_history[2:])
        if isinstance(handoff_message_data.input_history, tuple)
        else handoff_message_data.input_history
    )

    return HandoffInputData(
        input_history=history,
        pre_handoff_items=tuple(handoff_message_data.pre_handoff_items),
        new_items=tuple(handoff_message_data.new_items),
    )

In [2]:
# 定义三个 Agent
first_agent = Agent(
    name="Assistant",
    instructions="Be extremely concise.",
    tools=[random_number_tool],
)

spanish_agent = Agent(
    name="Spanish Assistant",
    instructions="You only speak Spanish and are extremely concise.",
    handoff_description="A Spanish-speaking assistant.",
)

second_agent = Agent(
    name="Assistant",
    instructions=(
        "Be a helpful assistant. If the user speaks Spanish, handoff to the Spanish assistant."
    ),
    handoffs=[handoff(spanish_agent, input_filter=spanish_handoff_message_filter)],
)

## 2. Agent 结构可视化

使用 SDK 的 `draw_graph` 可视化 Agent 与 Handoff 关系。

**依赖安装（需两步）：**
1. Python 包：`pip install "openai-agents[viz]"`
2. **系统 Graphviz**（`dot` 可执行文件）：
   - Ubuntu/Debian: `sudo apt-get install graphviz`
   - macOS: `brew install graphviz`

In [13]:
try:
    from agents.extensions.visualization import draw_graph
    print("=== first_agent（带 random_number_tool）===")
    draw_graph(first_agent, filename="first_agent.png")
except ImportError as e:
    print("可视化需要安装: pip install \"openai-agents[viz]\"")
    print(f"错误: {e}")
except Exception as e:
    if type(e).__name__ == "ExecutableNotFound":
        print("系统未安装 Graphviz，请执行: sudo apt-get install graphviz  (Ubuntu/Debian)")
    else:
        raise

=== first_agent（带 random_number_tool）===


In [12]:
try:
    from agents.extensions.visualization import draw_graph
    print("=== second_agent（Handoff 到 Spanish Assistant）===")
    draw_graph(second_agent, filename="second_agent")
except ImportError as e:
    print("可视化需要安装: pip install \"openai-agents[viz]\"")
    print(f"错误: {e}")
except Exception as e:
    if type(e).__name__ == "ExecutableNotFound":
        print("系统未安装 Graphviz，请执行: sudo apt-get install graphviz  (Ubuntu/Debian)")
    else:
        raise

=== second_agent（Handoff 到 Spanish Assistant）===


In [5]:
# 可选：保存图为 PNG 文件
# draw_graph(second_agent, filename="handoff_graph")

## 3. 运行 Handoff 工作流

执行完整对话流程，Step 4 会触发 handoff 到 Spanish Assistant。

In [6]:
with trace(workflow_name="Handoff message filter"):
    # Step 1: 发送第一条消息
    result = await Runner.run(first_agent, input="Hi, my name is Sora.")  # type: ignore[top-level-await]
    print("Step 1 done")

    # Step 2: 请求生成随机数
    result = await Runner.run(
        first_agent,
        input=result.to_input_list()
        + [{"content": "Can you generate a random number between 0 and 100?", "role": "user"}],
    )
    print("Step 2 done")

    # Step 3: 切换到 second_agent
    result = await Runner.run(
        second_agent,
        input=result.to_input_list()
        + [
            {
                "content": "I live in New York City. Whats the population of the city?",
                "role": "user",
            }
        ],
    )
    print("Step 3 done")

    # Step 4: 触发 handoff（用户说西班牙语）
    result = await Runner.run(
        second_agent,
        input=result.to_input_list()
        + [
            {
                "content": "Por favor habla en español. ¿Cuál es mi nombre y dónde vivo?",
                "role": "user",
            }
        ],
    )
    print("Step 4 done")

[non-fatal] Tracing: request failed: [Errno 101] Network is unreachable
[non-fatal] Tracing: request failed: [Errno 101] Network is unreachable
Error getting response: Request timed out.. (request_id: None)
Error getting response; filtered.input=[{'content': 'Hi, my name is Sora.', 'role': 'user'}]


APITimeoutError: Request timed out.

[non-fatal] Tracing: request failed: [Errno 101] Network is unreachable
[non-fatal] Tracing: max retries reached, giving up on this batch.
[non-fatal] Tracing: request failed: [Errno 101] Network is unreachable
[non-fatal] Tracing: request failed: [Errno 101] Network is unreachable
[non-fatal] Tracing: request failed: [Errno 101] Network is unreachable
[non-fatal] Tracing: max retries reached, giving up on this batch.


## 4. 查看最终消息

由于 input_filter 生效，Spanish Assistant 收到的历史已过滤：前两条消息被移除，工具调用也被移除。

In [ ]:
print("\n=== Final messages ===\n")
for item in result.to_input_list():
    print(json.dumps(item, indent=2, ensure_ascii=False))